In [ ]:
import requests
from bs4 import BeautifulSoup
import datetime
from datetime import datetime
!pip install openpyxl
import openpyxl
wb = openpyxl.Workbook()
sheet = wb.active
sheet.append(["날짜","제목","본문"])
for i in range(1,101): # 마지막 페이지 입력하기 101
    URL = 'https://finance.naver.com/research/debenture_list.nhn?keyword=&brokerCode=&searchType=writeDate&writeFromDate=2011-01-01&writeToDate=2021-12-31&x=0&y=0&page='+str(i)
    response = requests.get(URL)
    soup = BeautifulSoup(response.text, "html.parser")
    list_minutes = soup.select('div[id="contentarea_left"] div[class="box_type_m"] tr')
    for minutes in list_minutes:
        try:
            title=minutes.select('a')[0].text
            sub_link = minutes.select('a')[1].attrs['href']
            date = datetime.strptime('20'+ minutes.select('.date')[0].text, "%Y.%m.%d")
            if str(sub_link).endswith('.pdf'):
                print(date.strftime('%Y.%m.%d'))
                print(title)
                print(sub_link)
                file_res = requests.get(sub_link)
                sheet.append([ds.strftime('%Y.%m.%d'),title,sub_link])
            else:
                pass
        except:
            pass
wb.save("bond_report.csv")

In [ ]:
# option 2 - x download pdf/ getting urls only
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import csv
f = open('bond_report.csv','w',newline='')
wr = csv.writer(f)
for i in range(1,102): # 마지막 페이지+1 입력
    URL = 'https://finance.naver.com/research/debenture_list.nhn?keyword=&brokerCode=&searchType=writeDate&writeFromDate=2011-01-01&writeToDate=2021-12-31&x=0&y=0&page='+str(i)
    response = requests.get(URL)
    soup = BeautifulSoup(response.text, "html.parser")
    list_minutes = soup.select('div[id="contentarea_left"] div[class="box_type_m"] tr')
    for minutes in list_minutes:
        try:
            title=minutes.select('a')[0].text
            sub_link = minutes.select('a')[1].attrs['href']
            date = datetime.strptime('20'+ minutes.select('.date')[0].text, "%Y.%m.%d")
            date_convert = date.strftime('%Y.%m.%d')
            print(date_convert)
            print(title)
            print(sub_link)
            wr.writerow([date_convert,title,sub_link])
        except:
            pass
f.close()

In [ ]:
#링크안 pdf text 긁어오기
import os
from tika import parser
import pandas as pd
import csv
bond_report = pd.read_csv("bond_report.csv", header=0)
bond_report.columns = ['date','title','url']
content = []
for url in bond_report['url']:
    try:
        content.append(parser.from_file(url)['content'].replace('\n', ''))
    except:
        content.append("NA")
        pass
bond_report['content'] = content
bond_report 
bond_report.to_csv(r'/Users/soojinkim/kdigital/mini_project_1/bond_report_df.csv')